In [ ]:
from math import log2
import essentia.standard
import numpy as np
from collections import Counter
import os
from time import time
from collections import OrderedDict
from random import sample
from itertools import chain

In [ ]:
melodies_path = 'dataset/chromas_hpcp/vocals_spleeter/'
#melodies_path = 'dataset/chromas_hpcp/vocals_demucs/'
#melodies_path = 'dataset2/chromas_hpcp/vocals_spleeter/'
#melodies_path = 'dataset2/chromas_hpcp/vocals_demucs/'
hummings_path = 'dataset/hummings/'
#hummings_path = 'dataset2/hummings/'
melodies = os.listdir(melodies_path)
hummings = os.listdir(hummings_path)

In [ ]:
def compute_similarity(_chroma, _chroma_db):
    _crp = essentia.standard.ChromaCrossSimilarity()
    _crp_matrix = _crp(_chroma, _chroma_db)
    _score_matrix, _distance = essentia.standard.CoverSongSimilarity()(_crp_matrix)
    return _distance

In [ ]:
def runSample(_hummings, _melodies):
    numerador_MRR = 0
    numerador_TOP20 = 0
    numerador_TOP15 = 0
    numerador_TOP10 = 0
    numerador_TOP5 = 0
    numerador_TOP1 = 0
    denominador = 0
    tiempo = 0

    for hum in _hummings:
        denominador += 1
        results = {}
        start_time=time()
        loader = essentia.standard.EqloudLoader(filename = hummings_path + hum, sampleRate=44100)
        audio = loader()
        pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=180)
        pitch_values, _ = pitch_extractor(audio)
        chroma = hpcpgram(pitch_values, sampleRate=44100, hopSize=50)
        for melody in _melodies:
            chroma_db = np.loadtxt(melodies_path + melody)
            results[melody] = compute_similarity(chroma_db, chroma)
        ordered_results = OrderedDict({k: v for k, v in sorted(results.items(), key=lambda item: item[1])})
        tiempo += (time()-start_time)
        for i, rank in enumerate(ordered_results.items()):
            if hum.split('.')[0].split('+')[0] == rank[0].split('.')[0]:
                numerador_MRR += (1/(i+1))
                if (i+1) <= 1:
                    numerador_TOP1 += 1
                if (i+1) <= 5:
                    numerador_TOP5 += 1
                if (i+1) <= 10:
                    numerador_TOP10 += 1
                if (i+1) <= 15:
                    numerador_TOP15 += 1
                if (i+1) <= 20:
                    numerador_TOP20 += 1

    mrr = numerador_MRR / denominador
    top1 = numerador_TOP1 / denominador * 100
    top5 = numerador_TOP5 / denominador * 100
    top10 = numerador_TOP10 / denominador * 100
    top15 = numerador_TOP15 / denominador * 100
    top20 = numerador_TOP20 / denominador * 100
    tiempo = tiempo / denominador

    print('MRR: ', mrr)
    print('Top-1: ', top1)
    print('Top-5: ', top5)
    print('Top-10: ', top10)
    print('Top-15: ', top15)
    print('Top-20: ', top20)
    print('Tiempo: ', tiempo)
    
    return [mrr, top1, top5, top10, top15, top20]

In [ ]:
count_by_humming = {}
for humming in hummings:
    count_by_humming[humming.split('.')[0].split('+')[0]] = 0
for humming in hummings:
    count_by_humming[humming.split('.')[0].split('+')[0]] += 1

In [ ]:
by_qty = {}

for index, key in enumerate(count_by_humming):
    qty = count_by_humming[key]
    if qty in by_qty:
        by_qty[qty].append(key)
    else:
        by_qty[qty] = [key]

In [ ]:
def getSamples():
    hummings_sample = []

    hummings_sample.append(sample(by_qty[1], 10))
    hummings_sample.append(sample(by_qty[2], 4))
    hummings_sample.append(sample(by_qty[3], 2))

    # The Flattening (Flatten the list of lists into a single level list)
    hummings_sample = list(chain.from_iterable(hummings_sample))

    final_humming_list = []

    for hum in hummings:
        if hum.split('.')[0].split('+')[0] in hummings_sample:
            final_humming_list.append(hum)

    filtered_melodies = []

    for melody in melodies:
        if (melody.split('.')[0]) not in hummings_sample:
            filtered_melodies.append(melody)

    remaining_songs_count = 50 - len(hummings_sample)

    final_song_list = []

    for hum in hummings_sample:
        file_name = hum + '.txt'
        final_song_list.append(file_name)

    additional_random_songs = sample(filtered_melodies, remaining_songs_count)

    final_song_list = final_song_list + additional_random_songs
    
    return [final_humming_list, final_song_list]

In [ ]:
avg_mrr = 0
avg_top1 = 0
avg_top5 = 0
avg_top10 = 0
avg_top15 = 0
avg_top20 = 0

for i in range(5):
    selected_hummings, selected_melodies = getSamples()

    [_mrr, _top_1, _top_5, _top_10, _top_15, _top_20] = runSample(selected_hummings, selected_melodies)

    avg_mrr += _mrr
    avg_top1 += _top_1
    avg_top5 += _top_5
    avg_top10 += _top_10
    avg_top15 += _top_15
    avg_top20 += _top_20
    
    print("================================")
    print("================================")
    print("================================")

print('avg mrr: ', avg_mrr/5)
print('avg top1: ', avg_top1/5)
print('avg top5: ', avg_top5/5)
print('avg top10: ', avg_top10/5)
print('avg top15: ', avg_top15/5)
print('avg top20: ', avg_top20/5)